In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


import ipykernel
import pandas as pd

#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)


In [2]:
import json
import random
import matplotlib.pyplot as plot
import numpy as np
import os
import jsonlines
import ast
import time, os, fnmatch, shutil
from collections import defaultdict
import matplotlib
import requests

import math
import random
import copy

from collections import namedtuple, deque

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

from games  import games
from games import games
from game   import Game
from player import Player_Template
from team   import Team
from userinput import userInput



HISTORY_FILE = None

summary_dict = {"losses":0,"wins":0,"draws":0}

cycle = 0

actions = [[], []]
rewards = [[], []]
history = {"cycle": cycle, "actions": actions, "rewards": rewards, "value": 0}

score_board = {}
#step_action_dict = defaultdict()
#step_action_dict['random_moves'] = defaultdict()
#step_action_dict['policy_moves'] = defaultdict()

numeric_names = {'w_R0': 1, 'w_K0': 2, 'w_B0': 3, 'w__K': 4, 'w__Q': 5, 'w_B1': 6, 'w_K1': 7, 'w_R1': 8, 'w_P0': 9, 'w_P1':
 10, 'w_P2': 11, 'w_P3': 12, 'w_P4': 13, 'w_P5': 14, 'w_P6': 15, 'w_P7': 16, 'b_P0': -16, 'b_P1': -15, 'b_P2': -14, 'b_P3':
 -13, 'b_P4': -12, 'b_P5': -11, 'b_P6': -10, 'b_P7': -9, 'b_R0': -8, 'b_K0': -7, 'b_B0': -6, 'b__K': -5, 'b__Q': -4, 'b_B1'
: -3, 'b_K1': -2, 'b_R1': -1}




In [3]:
%%writefile sparse_action_dict.py

sparse_action_dict = {'1,1,0':'010010011001','2,1,-2':'010011010111','2,1,2':'010011011011','2,2,-1':'010011100000','2,2,1':'010011100010','3,1,-1':'010100011000','3,1,1':'010100011010','4,1,-1':'010101011000','4,1,0':'010101011001','4,1,1':'010101011010','5,1,-1':'010110011000','5,1,0':'010110011001','5,1,1':'010110011010','6,1,-1':'010111011000','6,1,1':'010111011010','7,1,-2':'011000010111','7,1,2':'011000011011','7,2,-1':'011000100000','7,2,1':'011000100010','8,1,0':'011001011001','9,1,0':'011010011001','9,1,-1':'011010011000','9,1,1':'011010011010','10,1,0':'011011011001','10,1,-1':'011011011000','10,1,1':'011011011010','11,1,0':'011100011001','11,1,-1':'011100011000','11,1,1':'011100011010','12,1,0':'011101011001','12,1,-1':'011101011000','12,1,1':'011101011010','13,1,0':'011110011001','13,1,-1':'011110011000','13,1,1':'011110011010','14,1,0':'011111011001','14,1,-1':'011111011000','14,1,1':'011111011010','15,1,0':'100000011001','15,1,-1':'100000011000','15,1,1':'100000011010','16,1,0':'100001011001','16,1,-1':'100001011000','16,1,1':'100001011010','-16,-1,0':'000001001001','-16,-1,-1':'000001001000','-16,-1,1':'000001001010','-15,-1,0':'000010001001','-15,-1,-1':'000010001000','-15,-1,1':'000010001010','-14,-1,0':'000011001001','-14,-1,-1':'000011001000','-14,-1,1':'000011001010','-13,-1,0':'000100001001','-13,-1,-1':'000100001000','-13,-1,1':'000100001010','-12,-1,0':'000101001001','-12,-1,-1':'000101001000','-12,-1,1':'000101001010','-11,-1,0':'000110001001','-11,-1,-1':'000110001000','-11,-1,1':'000110001010','-10,-1,0':'000111001001','-10,-1,-1':'000111001000','-10,-1,1':'000111001010','-9,-1,0':'001000001001','-9,-1,-1':'001000001000','-9,-1,1':'001000001010','-8,-1,0':'001001001001','-7,-1,-2':'001010000111','-7,-1,2':'001010001011','-7,-2,-1':'001010000000','-7,-2,1':'001010000010','-6,-1,-1':'001011001000','-6,-1,1':'001011001010','-5,-1,-1':'001100001000','-5,-1,0':'001100001001','-5,-1,1':'001100001010','-4,-1,-1':'001101001000','-4,-1,0':'001101001001','-4,-1,1':'001101001010','-3,-1,-1':'001110001000','-3,-1,1':'001110001010','-2,-1,-2':'001111000111','-2,-1,2':'001111001011','-2,-2,-1':'001111000000','-2,-2,1':'001111000010','-1,-1,0':'010000001001'}

Overwriting sparse_action_dict.py


In [4]:
device  = torch.device("cuda:1" if torch.cuda.is_available() else "cuda:0")
device

device(type='cuda', index=1)

In [5]:
args = { 
    "BUFFER_SIZE":int(1e6),
    "BATCH_SIZE":1024,  
    "GAMMA":0.99,
    "TAU":2e-3,
    "LR_ACTOR":1e-3,
    "LR_CRITIC":1.1e-3,
    "WEIGHT_DECAY":0.0001,
    "UPDATE_EVERY":5,
    "EXPLORE_NOISE":0.05,
    "FC1_UNITS":4096,
    "FC2_UNITS":10240,
    "FC3_UNITS":1024,
    "seed":0,
    "state_size":384,
    "action_size":1,
    "num_agents":2,
    "device":device,
    'mcritic_path':'/home/ubuntu/chessy/checkpoint_mCritic.pth',
    'agent_p0_path':'/home/ubuntu/chessy/checkpoint_p0.pth',
    'agent_p1_path':'/home/ubuntu/chessy/checkpoint_p1.pth'
}

In [6]:
#sparse_action_dict

In [7]:
initial_state =  "010010010011010100010101010110010111011000011001011010011011011100011101011110011111100000100001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000010000011000100000101000110000111001000001001001010001011001100001101001110001111010000"

In [8]:
best_moves   = 0
policy_moves = 0
random_moves = 0

In [9]:
def getFunc(piece):
    def piece_func():
        return piece
    return piece_func()

def Bishop():
    return getFunc("Bishop")

def Pawn():
    return getFunc("Pawn")

def King():
    return getFunc("King")

def Queen():
    return getFunc("Queen")

def Knight():
    return getFunc("Knight")

def Rook():
    return getFunc("Rook")


In [10]:

class Incorrect_Input_error(Exception):
  """Generic input error handler: raised in the case that any of the user inputed data is incorrect"""
  pass



In [11]:
def run_trial(user_input, env):
    global cycle
    for j in range(user_input.num_sides):
        team_name = "team_" + str(j)
        team_name = Team(user_input.game, user_input.teams[j]["team_name"], j,user_input.teams[j]["skill"] / 10, user_input.teams[j]["strategy"])

        env.insert_team(team_name)
            
    env.not_deadlocked = True
    env.states.append(initial_state)
    time_step = 0
        
    while env.not_deadlocked:
        state_prior = env.states[-1] 
        env.step(cycle, user_input)
        next_state = ""
        state_prime = ""
        if time_step % 2 == 0:
            state_prime = env.states[-1]
        else:
            next_state = env.states[-1]
        #print('FEASIBLE_MOVES:\n{}\n'.format(env.feasible_moves_))
        #print("CYCLE:\t{}\nSTATE_PRIOR:\t{}\nACTION:\t{}\nNEXT_STATE:\t{}\nREWARD:\t{}\n".format(cycle,state_prior,env.last_action_sparse, next_state, env.last_reward))
            
        time_step += 1

In [12]:
def run_trials(user_input):
    """ Runs the num_trials """
    global cycle
    global HISTORY_FILE
    global SUMMARY_FILE
    
    print("==============================RUN_TRIALS==================================")
    
    t = time.localtime()
    timestamp = time.strftime('%b_%d_%Y_%H%M', t)
    num_trials = user_input.num_trials
    num_sides = user_input.num_sides
    
    SUMMARY_FILE = ("/data_data/reinforcement_learning/results/summary_file.tsv")

    HISTORY_FILE = ("/data_data/reinforcement_learning/results/history_file_" + str(num_trials) + "_trials_" + str(num_sides) + "_sides_" + str(timestamp))
    
    """ Clear output file """
    
    with open(HISTORY_FILE, "w") as history_file:
        history_file.write("\n")
        
        
    for i in range(num_trials):
        
        sides = [x for x in range(num_sides)]
        
        env = Game(user_input.game, 8, sides, user_input.display_board_positions)

        run_trial(user_input,env)

        cycle += 1
    
   


In [13]:
args['action_size']

1

In [14]:
from agent import Agent
from model import ActorNetwork, CriticNetwork, MCritic
from replaybuffer import ReplayBuffer

mCritic       =  MCritic(args['state_size'],args['action_size'], args)
agent_p0      =  Agent(args['state_size'],args['action_size'], 0,args)
agent_p1      =  Agent(args['state_size'],args['action_size'], 1,args)

In [15]:
print(mCritic.network)
print(agent_p0.actor_network)

CriticNetwork(
  (fc1): Linear(in_features=385, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=10240, bias=True)
  (fc3): Linear(in_features=10240, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=1, bias=True)
)
ActorNetwork(
  (fc1): Linear(in_features=384, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=10240, bias=True)
  (fc3): Linear(in_features=10240, out_features=1024, bias=True)
  (fc4): Linear(in_features=1024, out_features=1, bias=True)
)


In [16]:
args['action_size']

1

In [17]:
test_state = torch.IntTensor(np.random.choice((0,1),args['state_size'])).to(device)
test_state

tensor([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
        1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
        0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
        0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
        1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
        1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
        1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
        0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0,
        1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,

In [18]:
test_action = torch.IntTensor([34]).to(device)
test_action[:25]

tensor([34], device='cuda:1', dtype=torch.int32)

In [19]:
test_action = torch.IntTensor(np.array([34]).reshape((1,))).to(device)
test_action[:25]

tensor([34], device='cuda:1', dtype=torch.int32)

In [20]:
x = torch.cat((test_state, test_action))
print(x[:25])

tensor([0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
        1], device='cuda:1', dtype=torch.int32)


In [21]:
test_state.shape

torch.Size([384])

In [22]:
agent_p0.actor_network(test_state.float())

/home/ubuntu/chessy/model.py:46: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(self.fc4(x))


tensor([0.], device='cuda:1', grad_fn=<LogSoftmaxBackward>)

In [23]:
test_action

tensor([34], device='cuda:1', dtype=torch.int32)

In [26]:
mCritic.network(test_state.float(), test_action.float())

tensor([-0.0214], device='cuda:1', grad_fn=<AddBackward0>)

In [ ]:
torch.cat((test_state, test_action))

In [ ]:

from agent import Agent
from model import ActorNetwork, CriticNetwork, MCritic
from replaybuffer import ReplayBuffer

#def main():
if __name__ == "__main__":
    global cycle
    global HISTORY_FILE
    global score_board
    score_board[0] = []
    score_board[1] = []
    user_input = userInput()
    run_trials(user_input)


#main()


In [ ]:
!ls 

In [ ]:
print(numeric_names)

In [ ]:
Pieces = {
    'Pawn':("w_P","b_P"),
    'Knight':("w_K","b_K"),
    'King':("w__K","b__K"),
    'Queen':("w__Q","b__Q"),
    'Bishop':("w_B","b_B"),
    'Rook':("w_R","b_R"),
}
    

In [ ]:
Pieces_detail = defaultdict()
actions_array = []
sparse_action_dict = defaultdict()
actions = []

for piece in numeric_names.keys():
    action_sparse = []
    if piece[0] == 'w':
        idx = 0
    else:
        idx = 1
        
    if piece[1:3] == '_R':
        player_id = 'Rook'
    elif piece[1:3] == '_K':
        player_id = 'Knight'
    elif piece[1:3] == '_B':
        player_id = 'Bishop'
    elif piece[1:3] == '_P':
        player_id = 'Pawn'
    elif piece[1:4] == '__K':
        player_id = 'King'
    elif piece[:3] == '__Q':
        player_id = 'Queen'
        
    actions = games['chessy']['players'][player_id]['moves']
    
    actions = actions[idx]
    
    #action_diff = 4 - len(actions)
    
    #for i in range(action_diff):
    #    actions.append((0,0))
    
    piece_num_id = numeric_names[piece]
    
    piece_bin_id = '{0:06b}'.format(piece_num_id + 17)
    
    actions_ = []
    
    for action in actions:
        action_value_  = (8*action[0] + action[1] + 17)
        action_value   = '{0:06b}'.format(action_value_)
        actions_.append(action_value)
        
        piece_move_bin = (str(piece_bin_id)+str(action_value))
        
        actions_array.append(piece_move_bin)
        
        sparse_action  = str(piece_num_id)+","+str(action[0])+ "," +str(action[1])
        
        action_sparse.append(sparse_action)
        
        sparse_action_dict[sparse_action] = piece_move_bin
    
    print(piece_num_id, "\t",piece_bin_id, "\t", action_sparse,  "\t", actions, "\t", actions_)
    
    Pieces_detail[piece_num_id] = {
        'piece_num_id':piece_num_id,
        'piece_bin_id':piece_bin_id,
        'action_sparse':action_sparse,
        'actions_verbose': actions,
        'actions_bin': actions_
       }
    
    

In [ ]:
action_ids = np.arange(0,len(sparse_action_dict.keys()),1)
action_ids

In [ ]:
action_id_dict = {y:x for x,y in zip(action_ids,sparse_action_dict.keys())}
action_id_dict 

In [ ]:
id_action_dict = {x:y for x,y in zip(action_ids,sparse_action_dict.keys())}
id_action_dict

In [ ]:
actions_array

In [ ]:
Pieces_detail

In [ ]:
print(actions)

In [ ]:
  def getBin(num):
        if int(num) != 0:
            return "{0:{fill}6b}".format(int(num)+17, fill='0')
        else:
            return "{0:{fill}6b}".format(0, fill='0')

In [ ]:
'{0:06b}'.format(32)

In [ ]:
getBin(1)

In [ ]:
!cat games.py